In [1]:
from fsds_100719.imports import *

fsds_1007219  v0.7.21 loaded.  Read the docs: https://fsds.readthedocs.io/en/latest/ 


Handle,Package,Description
dp,IPython.display,Display modules with helpful display and clearing commands.
fs,fsds_100719,Custom data science bootcamp student package
mpl,matplotlib,Matplotlib's base OOP module with formatting artists
plt,matplotlib.pyplot,Matplotlib's matlab-like plotting module
np,numpy,scientific computing with Python
pd,pandas,High performance data structures and tools
sns,seaborn,High-level data visualization library based on matplotlib


[i] Pandas .iplot() method activated.


In [2]:
import os
os.listdir('ml-latest-small/')

['links.csv', 'tags.csv', 'ratings.csv', 'README.txt', 'movies.csv']

In [3]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [54]:
ratings = pd.read_csv('ml-latest-small/ratings.csv',)
ratings.drop('timestamp',axis=1,inplace=True)
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [55]:
links = pd.read_csv('ml-latest-small/links.csv')
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [56]:
tags = pd.read_csv('ml-latest-small/tags.csv')
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [51]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [62]:
# !pip install surprise
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split

reader = Reader()
data = Dataset.load_from_df(ratings,reader)

In [64]:
trainset, testset = train_test_split(data, test_size=.25)


## Collaborative Filtering

Collaborative filtering is a method of making automatic predictions (i.e. filtering) about the interests of a user by collecting preferences or taste information from many users on the aggregate (i.e. collaborating). There are two main approaches to collaborative filtering that we will learn about. The basic idea behind collaborative filtering model is:

- Predict a numerical value expressing the predicted score of an item for a user. The predicted value should be  within the same scale that is used by all users for the rating (i.e. the number of stars or rating between 0-5) 

- Recommend a list of Top-N items that the active user will like the most based on the highest predicted ratings for the items that they have not yet seen

Here is a representation of that last bullet point:  

<img src = "https://raw.githubusercontent.com/learn-co-students/dsc-collaborative-filtering-singular-value-decomposition-online-ds-pt-100719/master//images/making_recommendations.png">  

When dealing with utility matrices, there are two different ways to go about determining similarity within the utility matrix. 


* Item-based: measure the similarity between the items that target users rates/interacts with and other items
* User-based: measure the similarity between target users and other users

Before we dive into the differences between these two methods, let's look at what these similarity metrics are, and how they are related to the final score prediction.

### Similarity Metrics

**Pearson Correlation**: This similarity metric only takes into account those items that are rated by both individuals.

$$ \text{pearson correlation}(u,v) = \frac{\sum_{i \in I_{uv}}{(r_{ui}- \mu_{u})*(r_{vi}- \mu_{v})}}{\sqrt{\sum_{i \in I_{uv} }{(r_{ui}-\mu_{u})^{2}  }}  * \sqrt{\sum_{i \in I_{uv} }{(r_{vi}-\mu_{v})^{2}  }}} $$


**Cosine Similarity**: Determines how vectors are related to each other by measuring the cosine angle between two vectors. The value also ranges from [-1, 1], with -1 meaning that the two vectors are diametrically opposed, 0 meaning the two vectors are perpendicular to one another, and 1 meaning that the vectors are the same. Here is the formula in the context of user similarity:

$$ \text{cosine similarity}(u,v) = \frac{\sum_{i \in I_{uv}}{r_{ui}*r_{vi}}}{\sqrt{\sum_{i \in I_{uv} }{r_{ui}^{2}  }}  * \sqrt{\sum_{i \in I_{uv} }{r_{ui}^{2}  }}} $$

where $u$ is a user and $v$ is another user being compared to $u$. $i$ represents each item being rated. $I$ is the entire item set.


**Jaccard Similarity**: Uses the number of preferences in common between two users into account. Importantly, it does not take the actual values of the ratings into account, only whether or not users have rated the same items. In other words, all explicit ratings are effectively turned into values of 1 when using the Jaccard Similarity metric.


$$ \text{Jaccard Similarity}(u,v) = \frac{I_{u} \cap I_{v}}{I_{u} \cup I_{v}}$$

- Two main types
    1. Memory-Based/Neighborhood-Based
    2. Model-Based 

## Singular Value Decomposition

__Singular-Value Decomposition__ or SVD is a common and widely used matrix decomposition method. All matrices are able to be factored using an SVD, which makes it more stable than other methods, such as the eigendecomposition. As such, it is often used in a wide array of applications including compression and data reduction.

In simple terms, SVD is the factorization of a matrix into 3 matrices. So if we have a matrix A, then its SVD is represented by the equation:

$$ A = U\Sigma V^T$$

Where $A$ is an $n x d$ matrix, $U$ is an $(n x r)$ orthogonal matrix, $𝚺$ is an $(r x r)$ nonnegative rectangular diagonal matrix, and $V$ is an $(r x d)$ orthogonal matrix.
$U$ is also referred to as the __left singular vectors__, 𝚺 the __singular values__, and V the __right singular vectors__. 

In [38]:
from surprise import SVD
from surprise.model_selection import cross_validate
algo = SVD()
cross_validate(algo, data,measures=['RMSE','MAE'],cv=3,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8758  0.8829  0.8839  0.8808  0.0036  
MAE (testset)     0.6761  0.6785  0.6790  0.6779  0.0013  
Fit time          3.13    3.08    3.11    3.10    0.02    
Test time         0.18    0.32    0.15    0.22    0.07    


{'test_rmse': array([0.87578668, 0.88287816, 0.88386717]),
 'test_mae': array([0.67609444, 0.6784507 , 0.67901803]),
 'fit_time': (3.131141185760498, 3.0759541988372803, 3.1061289310455322),
 'test_time': (0.17903494834899902, 0.32342100143432617, 0.1544780731201172)}

In [39]:
algo=SVD()
algo.fit(trainset)
predictions = algo.test(testset)

In [40]:
from surprise import accuracy,similarities

In [41]:
accuracy.rmse(predictions)

RMSE: 0.8743


0.8742749745690356